<a href="https://colab.research.google.com/github/marianagg0412/Analisis/blob/main/Sistemas_de_ecuaciones_lineales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sistemas de Ecuaciones Lineales**

In [1]:
import numpy as np
eps=np.finfo(float).eps #Esto es para encontrar el épsilon de la máquina

In [2]:
def DLU(A):
  filas = len(A)
  columnas = len(A[0]) #A una matriz rectangular la puedo descomponer de la forma D,L,U

  # Crea tres matrices llenas de ceros
  D = np.zeros((filas,columnas))
  L = np.zeros((filas,columnas))
  U = np.zeros((filas,columnas))

  # Copia cada elemento ij en la matriz correspondiente D,L,U
  for i in range(filas):
    for j in range(columnas):
      if i==j:
        D[i][j] = A[i][j]
      if i<j:
        U[i][j] = -1*A[i][j]
      if i>j:
        L[i][j] = -1*A[i][j]
  return(D,L,U)

In [9]:
def matriz_A():
  A = np.array([[2,3,4,5],[3,6,2,5],[4,3,-10,5],[1,1,0,3]])
  return A

In [11]:
A = matriz_A()

In [12]:
DLU(A)

(array([[  2.,   0.,   0.,   0.],
        [  0.,   6.,   0.,   0.],
        [  0.,   0., -10.,   0.],
        [  0.,   0.,   0.,   3.]]),
 array([[ 0.,  0.,  0.,  0.],
        [-3.,  0.,  0.,  0.],
        [-4., -3.,  0.,  0.],
        [-1., -1.,  0.,  0.]]),
 array([[ 0., -3., -4., -5.],
        [ 0.,  0., -2., -5.],
        [ 0.,  0.,  0., -5.],
        [ 0.,  0.,  0.,  0.]]))

In [13]:
def vector_inicial():
  x0 = np.zeros((4,1))
  return x0

In [ ]:
vector_inicial()

In [14]:
def vector_b():
  b = np.array([[-1],[-2],[1],[0]])
  return b

In [ ]:
vector_b()

# Función del radio espectral


In [40]:
def radio_espectral(w):
  A = matriz_A()
  B = DLU(A)
  D = B[0]
  L = B[1]
  U = B[2]

  #T = np.linalg.inv(D)@(L+U) #Matriz Jacobi
  #T = np.linalg.inv(D - L)@(U) #Matriz Gauss-Seidel
  T =  np.linalg.inv(D - w*L)@((1-w)*D + w*U) #Matriz SOR
  v = np.linalg.eigvals(T)
  n = len(v)

  norma = np.array([ ])

  for i in range(n):
    norma_valor_propio = np.linalg.norm([v[i]])
    norma = np.append(norma, norma_valor_propio)

  radioEspectral = np.max(norma)
  return radioEspectral


In [37]:
def min_w():
    limit = 2  # Upper limit for w, you can adjust this as needed
    step = 0.01  # Step size for w
    finVal = float('inf')  # Initialize with positive infinity
    finW = None

    for w in [i * step for i in range(1, int(limit/step))]:  # Iterate over the range of w
        # Calculate the spectral radius for the SOR method using w
        spectral_radius = radio_espectral(w)  # You need to define this function
        if(spectral_radius < finVal):
          finVal = spectral_radius
          finW = w

    return finVal, finW  # Return the smallest spectral radius found

finVal, finW = min_w()
print("Radio espectral: ", finVal)
print("W mas optimo: ", finW)

Radio espectral:  0.5470998937796798
W mas optimo:  1.24


##Ejemplo

In [39]:
radio_espectral(0) #Was evaluated with Jacobi, w nonexistent

0.8465538327318075

# Jacobi

In [42]:
def jacobi(tolerancia, delta, Nmax): #Nmax will be 1000
  i = 0
  d = delta + 1
  error = tolerancia + 1

  A = matriz_A()
  b = vector_b()
  x0 = vector_inicial()
  B = DLU(A) #Evaluating coeficient matrix
  D = B[0]
  L = B[1]
  U = B[2]
  T = np.linalg.inv(D)@(L+U) #Jacobi method matrix
  C = np.linalg.inv(D)@(b) #Jacobi method vector

  while (error > tolerancia) and (delta < d) and (Nmax > i):
    i+=1
    print("Contador: ", i)
    x = T@x0 + C
    print("X ", x)
    d = np.linalg.norm(A@x - b)
    print("Delta: ", d)
    error = np.linalg.norm(x-x0)
    #error = np.linalg.norm(x-x0)/np.linalg.norm(x)
    print("Error: ", error) #Generally the error and the delta will not be the same.
    x0 = x
    print("-------------------------------------")

##Ejemplo

In [ ]:
jacobi(eps,eps,1000)

# Gauss Seidel

In [44]:
def gauss(tolerancia, delta, Nmax): #Nmax will be 1000 , Gauss-Seidel
  i = 0
  d = delta + 1
  error = tolerancia + 1

  A = matriz_A()
  b = vector_b()
  x0 = vector_inicial()
  B = DLU(A) #Evaluating coeficient matrix
  D = B[0]
  L = B[1]
  U = B[2]
  T = np.linalg.inv(D-L)@(U) #Gauss-Seidel method matrix
  C = np.linalg.inv(D-L)@(b) #Gauss-Seidel method vector

  while (error > tolerancia) and (delta < d) and (Nmax > i):
    i+=1
    print("Contador: ", i)
    x = T@x0 + C
    print("X ", x)
    d = np.linalg.norm(A@x - b)
    print("Delta: ", d)
    error = np.linalg.norm(x-x0)
    #error = np.linalg.norm(x-x0)/np.linalg.norm(x)
    print("Error: ", error) #Generally the error and the delta will not be the same.
    x0 = x
    print("-------------------------------------")

##Ejemplo

In [ ]:
gauss(eps,eps,1000)

# Método de SOR

In [6]:
def SOR(tolerancia, delta, Nmax, w): #Nmax will be 1000 , Gauss-Seidel
  i = 0
  d = delta + 1
  error = tolerancia + 1

  A = matriz_A()
  b = vector_b()
  x0 = vector_inicial()
  B = DLU(A) #Evaluating coeficient matrix
  D = B[0]
  L = B[1]
  U = B[2]
  T = np.linalg.inv(D-w*L)@((1-w)*D + w*U) #SOR method matrix
  C = w*np.linalg.inv(D-w*L)@(b) #SOR method vector

  while (error > tolerancia) and (delta < d) and (Nmax > i):
    i+=1
    print("Contador: ", i)
    x = T@x0 + C
    print("X ", x)
    d = np.linalg.norm(A@x - b)
    print("Delta: ", d)
    error = np.linalg.norm(x-x0) #Abolute
    #error = np.linalg.norm(x-x0)/np.linalg.norm(x) #Relative
    print("Error: ", error) #Generally the error and the delta will not be the same.
    x0 = x
    print("-------------------------------------")

##Ejemplo

In [19]:
radio_espectral(1.2) #Sacamos el w evaluando de a 0.1 -0.9 y 1.1-1.9

0.5963646867380821

In [47]:
result, minW = min_w()

print("Radio espectral: ",result)
print("Parameter w: ",minW)

Radio espectral:  0.5470998937796798
Parameter w:  1.24


In [46]:
SOR(eps,eps,1000,1.24)

Contador:  1
X  [[-0.62      ]
 [-0.02893333]
 [-0.4422832 ]
 [ 0.26822578]]
Delta:  2.3665160841276207
Error:  0.8079583124422398
-------------------------------------
Contador:  2
X  [[-0.15202158]
 [-0.4064922 ]
 [-0.07816985]
 [ 0.16647818]]
Delta:  1.2344915174053528
Error:  0.7102715063565836
-------------------------------------
Contador:  3
X  [[-0.14966044]
 [-0.36270298]
 [-0.21117985]
 [ 0.17182212]]
Delta:  0.5053970777263029
Error:  0.14015451821413819
-------------------------------------
Contador:  4
X  [[ 0.08162351]
 [-0.46715305]
 [-0.10008279]
 [ 0.1181149 ]]
Delta:  0.514984567333073
Error:  0.28218629142973656
-------------------------------------
Contador:  5
X  [[ 0.11136417]
 [-0.45094689]
 [-0.13926451]
 [ 0.11201328]]
Delta:  0.16227081124616743
Error:  0.0521496142839539
-------------------------------------
Contador:  6
X  [[ 0.19016862]
 [-0.48119502]
 [-0.10580919]
 [ 0.09340772]]
Delta:  0.1707050838908004
Error:  0.0926850412107853
----------------------